# Client understanding


In [2]:
# Libraries
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import collections 

# displays plots in the notebook
%matplotlib inline

def read_frame(name: str) -> pd.DataFrame:
    return pd.read_csv('../src/data/preprocessed/' + name + '.csv', sep=';')

In [3]:
client = read_frame("client")
client.head()

,client_id,district_id,birthdate_year,birthdate_month,birthdate_day,sex,birthdate
0,1,18,1970,12,13,f,1970-12-13
1,2,1,1945,2,4,m,1945-02-04
2,3,1,1940,10,9,f,1940-10-09
3,4,5,1956,12,1,m,1956-12-01
4,5,5,1960,7,3,f,1960-07-03


## Errors on the dataset
Here we want to find errors in the dataset. 

### Birthdates
Firstly let's search for invalid birthdates. For this let's check the possible year, months and days. 
As the code shows bellow, we don't have years such as 0000, thus let's accept the years as valid.
The same goes for the days and months.

In [4]:
def get_invalid_years(years: list) -> list:
    valid_years = range(1900, 2020)
    invalid_years = [i for i in years if i not in valid_years]
    return invalid_years

def get_invalid_months(months: list) -> list:
    valid_months = range(1, 13)
    invalid_month = [i for i in months if i not in valid_months]
    return invalid_month

def get_invalid_days(days: list) -> list: 
    invalid_days = range(1, 32)
    invalid_days = [i for i in days if i not in invalid_days] 
    return invalid_days

print(f"Number of invalid years {len(get_invalid_years(list(client['birthdate_year'])))}")
print(f"Number of invalid months {len(get_invalid_days(list(client['birthdate_month'])))}")
print(f"Number of invalid days {len(get_invalid_days(list(client['birthdate_day'])))}")

Number of invalid years 0
Number of invalid months 0
Number of invalid days 0


### Genres
Let's search for non binary genres. The genres are mostly equalized. 

In [5]:
print(client['sex'].value_counts())

m    2724
f    2645
Name: sex, dtype: int64


## Merge with loan

Before merge let's check if a person can own more than one account. 

In [6]:
def check_repeated(df: pd.DataFrame, table_name: str):
    count = df['account_id'].value_counts().values
    repeated = [i for i in count if i >= 2]
    print(f"Number of repeated values in  {table_name}: {len(repeated)}")


disp = read_frame("disp")
loan = read_frame("loan_dev")  

check_repeated(disp, "DISP") 
check_repeated(loan, "LOAN")

Number of repeated values in  DISP: 869
Number of repeated values in  LOAN: 0


Now we know that one account can have can be administrated by multiple clients. 

In [7]:

loan_merged = disp.merge(right = loan, how = "inner", on="account_id")
check_repeated(loan_merged, "LOAN_MERGED")

Number of repeated values in  LOAN_MERGED: 75


**PRESUPOSITION [1]**   
Prehaps the loans that are administrated by multiple clients are more capable of paying the loan.

In [8]:
repeated = loan_merged.groupby("loan_id").count()['disp_id'].sort_index()
df_repeated = pd.DataFrame({"loan_id": repeated.index, "count": repeated.values})
status = loan_merged[['loan_id', 'status']]
association = df_repeated.merge(right=status, on="loan_id", how="inner")
association.drop_duplicates(subset=['loan_id'])

,loan_id,count,status
0,4959,2,1
2,4961,1,-1
3,4973,1,1
4,4996,2,1
6,5002,2,1
...,...,...,...
396,7271,2,1
398,7284,1,1
399,7304,2,1
401,7305,1,1


Let's check our supposition by seeing the correlation between the variables

In [9]:
association[['count', 'status']].corr()

,count,status
count,1.000000,0.276395
status,0.276395,1.000000


**PRESUPOSITION [2]**  
Prehaps the genre inffluences how may not pay the debits.

In [10]:
client['sex'] = client['sex'].map({'m': 1, 'f': 0}, na_action=None)
client


,client_id,district_id,birthdate_year,birthdate_month,birthdate_day,sex,birthdate
0,1,18,1970,12,13,0,1970-12-13
1,2,1,1945,2,4,1,1945-02-04
2,3,1,1940,10,9,0,1940-10-09
3,4,5,1956,12,1,1,1956-12-01
4,5,5,1960,7,3,0,1960-07-03
...,...,...,...,...,...,...,...
5364,13955,1,1945,10,30,0,1945-10-30
5365,13956,1,1943,4,6,1,1943-04-06
5366,13968,61,1968,4,13,1,1968-04-13
5367,13971,67,1962,10,19,0,1962-10-19


In [11]:
loan_merged_2 = disp.merge(right = loan, how = "inner", on="account_id")
loan_merged_2 = loan_merged_2[['account_id', 'client_id', 'status']]
client_2 = client[['client_id', 'birthdate_year', 'sex']]
client_2

,client_id,birthdate_year,sex
0,1,1970,0
1,2,1945,1
2,3,1940,0
3,4,1956,1
4,5,1960,0
...,...,...,...
5364,13955,1945,0
5365,13956,1943,1
5366,13968,1968,1
5367,13971,1962,0


In [17]:
association = client_2.merge(right=loan_merged_2, on="account_id", how="inner")
association

KeyError: 'account_id'